<h3>Introduction</h3>

<h4> Background Information </h4>
A UBC Computer Science research group, led by Frank Wood, is collecting data regarding the ways in which different people play video games. The research group is using a Minecraft server that links to an external site in order to record the actions of each player as they use the server. The research group must make accommodations to manage the attracted players, ensuring they have sufficient resources such as software liscences and server hardware, while finding ways to target their recruitment efforts. The following investigation asks the predictive question, "Can age, player experience and average playing session time predict the time spent playing Minecraft?" and will address the question using a K-nearest neighbors predictive model.

<h4> Data Description </h4>
players_data (from players.csv): variables:

- experience: a character variable indicating the experience level of each player (outputs "Amateur", "Beginner", "Regular", "Veteran", or "Pro")
- subscribe: a logical variable indicating whether each player is subscribed to a game-related newsletter (outputs either "TRUE" or "FALSE")
- hashedEmail: a character variable containing information related to the players' emails
- played_hours: a double variable indicating how long each player used Minecraft in hours
- name: a character variable denoting each player's name
- gender: a character variable denoting each player's gender
- Age: a double variable denoting each player's age (as a whole number) number of observations: 196 (data was collected from 196 players)

Summary Statistics:

Subscription to newsletter

- "TRUE" = 144
- "FALSE" = 52

Age

- minimum age = 8 years
- maximum age = 50 years
- mean age = 20.52 years
- median = 19 years
- standard deviation = 6.174 years

Total Time Spent Playing Minecraft

- minimum time = 0.000 hours
- maximum time = 223.100 hours
- mean time = 5.846 hours
- median = 0.100 hours
- standard deviation = 28.357 hours

Issues: It seems that this data set is tidy. I can not predict any issues that may arise when analyzing the data in this data set.


sessions_data (from sessions.csv): variables:

- hashedEmail: a character variable containing information related to the players' emails
- start_time: a character variable denoting when a player started to play Minecraft (formatted as the date in DD/MM/YYYY followed by the time as HH:MM)
- end_time: a character variable denoting the time a player stopped playing Minecraft (formatted the same as start_time)
- original_start_time: a double character denoting the players' start times in UNIX time
- original_end_time: a double character denoting the players' end times in UNIX time number of observations: 1535

Summary Statistics: Original Start Time

- min = 1.712e+12 seconds
- max = 1.727e+12 seconds
- mean = 1.719e+12 seconds
- median = 1.719e+12 seconds
- standard deviation = 3.553e+9 seconds

Original End Time

- min = 1.712e+12 seconds
- max = 1.727e+12 seconds
- mean = 1.719e+12 seconds
- median = 1.719e+12 seconds
- standard deviation = 3.557e+9 seconds
- NA's = 2

Issues: the data set appears to be untidy since the start_time and end_time variables contain both a date and a time. To make the data tidy, the dates and times will need to be separated into different columns (columns for day, month, year, hour and minutes for start and end times). The original_start_time and original_end_time variables lack the necessary precision so will likely be irrelevant to the data analysis.